In [1]:
%load_ext autoreload
%autoreload 2
%autosave 10

gpu_id='gpu1'
import os
os.environ["THEANO_FLAGS"] = "device="+gpu_id +",lib.cnmem=1"

import numpy as np
from keras.layers.advanced_activations import SReLU
from keras import callbacks as ckbs
import random
import time
import gzip
import pandas as pd
from keras.models import model_from_json
import pickle as pkl
import keras
from keras.models import Sequential
from keras.layers.core import *
from keras.layers.wrappers import *
from keras.optimizers import *
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.layers import LSTM
#from docutils.languages.af import labels
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
from sklearn.grid_search import GridSearchCV, ParameterGrid

from sklearn.externals.joblib import load, dump
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, confusion_matrix
from keras.callbacks import EarlyStopping
from keras.models import load_model
from sklearn.metrics import accuracy_score


Autosaving every 10 seconds


Using Theano backend.
Using gpu device 1: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5105)
/home/sag/src_George/workspace/virtualenv/lib/python3.4/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
/home/sag/src_George/workspace/virtualenv/lib/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/sag/src_George/workspace/virtualenv/lib/python3.4/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This modu

In [ ]:
f = gzip.open('NerW2v.pkl', 'rb')
data = pkl.load(f, encoding='utf8')

In [ ]:
f = gzip.open('../data/embeddings_dial_ud_wiki.pkl.gz', 'rb')
embeddings = pkl.load(f)
f.close()

label2Idx = embeddings['label2Idx']
label_ud2Idx = embeddings['label_ud2Idx']
wordEmbeddings = np.array(embeddings['wordEmbeddings'])
caseEmbeddings = np.array(embeddings['caseEmbeddings'])

#Inverse label mapping
idx2Label = {v: k for k, v in label2Idx.items()}
idx_ud2Label = {v: k for k, v in label_ud2Idx.items()}

f = gzip.open('../data/data_dial_ud_wiki.pkl.gz', 'rb')
    
test_data_dial = pkl.load(f)
train_data_dial = pkl.load(f)
test_data_ud = pkl.load(f)
train_data_ud = pkl.load(f)
test_data_wiki = pkl.load(f)
train_data_wiki = pkl.load(f)



#test_data_dial = pkl.load(f)
#train_data_dial = pkl.load(f)

f.close()

In [ ]:
n_out = len(label_ud2Idx)
tokens = Sequential()
tokens.add(Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=True))

casing = Sequential()
casing.add(Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=True)) 
  
model = Sequential();
model.add(Merge([tokens, casing], mode='concat'))  
model.add(Bidirectional(LSTM(10, return_sequences=True, dropout_W=0.25))) 
model.add(Bidirectional(LSTM(10, return_sequences=True, dropout_W=0.35))) 
model.add(TimeDistributed(Dense(n_out, activation='softmax')))

sgd = SGD(lr=0.1, decay=1e-7, momentum=0.0, nesterov=False, clipvalue=3) 

model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd)

model.summary()


In [ ]:
tokens, casing, label = train_data[0]
#labels = [x[0] for x in label]
#labels = np.expand_dims([labels], -1)
print(label)

In [ ]:
def iterate_minibatches(dataset, startIdx, endIdx): 
    endIdx = min(len(dataset), endIdx)
    
    for idx in range(startIdx, endIdx):
        tokens, casing, label = dataset[idx] 
        #tokens, casing, label, char = dataset[idx]        
        labels = [x[0] for x in label]
        labels = np.expand_dims([labels], -1)     
        yield labels, np.asarray([tokens]), np.asarray([casing])#, np.asarray([char])


In [ ]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    #for tokens, casing, label, char in dataset:
    for tokens, casing, label in dataset:    
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        #char = np.asarray([char])
        #pred = model.predict_classes([tokens, casing, char], verbose=False)[0]
        pred = model.predict_classes([tokens, casing], verbose=False)[0] 
        labels = [x[0] for x in label]
        correctLabels.append(labels)
        predLabels.append(pred)
        
        
        
    return predLabels, correctLabels

In [ ]:
idx2Label

In [ ]:
def dialog_report(allINeed):

    tags = {2:"per",0:"loc",1:"org"}
    for fileName in list(set(allINeed["FileName"])):
        allObjects = []
        currentObject = []
        for cl in tags.keys():
            for i,row in allINeed.loc[allINeed["FileName"]==fileName].iterrows():
                if row["mark"]==cl:
                    if len(currentObject)>0:
                        lengthOfObj = int(row['posStartInText']) - int(currentObject[1]) + int(row['len'])
                        currentObject[2] = lengthOfObj
                    else:
                        currentObject.append(tags[cl])
                        currentObject.append(row['posStartInText'])
                        currentObject.append(row['len'])
                else:
                    if len(currentObject)>0:
                        currentObject[1]=str(int(currentObject[1]))
                        currentObject[2]=str(int(currentObject[2]))
                        allObjects.append(currentObject)
                        currentObject = []
            if len(currentObject)>0:
                currentObject[1]=str(int(currentObject[1]))
                currentObject[2]=str(int(currentObject[2]))
                allObjects.append(currentObject)
                currentObject = []
        allObjects = [" ".join(obj)+"\n" for obj in allObjects]
        WriteLinesTo(allObjects,"./forTests/"+fileName)

In [ ]:
def WriteLinesTo(allObjects, fileName):
    f = open(fileName, "w+")
    #print(len(allObjects))
    f.writelines(allObjects)
    f.close()

In [ ]:
list(idx2Label.values())

In [ ]:
label2Idx

In [ ]:
def train(train_data, test_data, model):
    import time
    number_of_epochs = 30
    stepsize = 24000
    print("%d epochs" % number_of_epochs)

    print("%d train sentences" % len(train_data))
    print("%d test sentences" % len(test_data))
    macc = 0.0
    for epoch in range(number_of_epochs):    
        print("--------- Epoch %d -----------" % epoch)
        random.shuffle(train_data)
        for startIdx in range(0, len(train_data), stepsize):
            start_time = time.time()    
            for batch in iterate_minibatches(train_data, startIdx, startIdx+stepsize):
                #labels, tokens, casing, char = batch
                labels, tokens, casing = batch
                #print(tokens.shape)
                #print(labels.shape)
                model.train_on_batch([tokens, casing], labels)
                #model.train_on_batch([tokens, casing, char], labels)   
            print("%.2f sec for training" % (time.time() - start_time))
            predLabels, correctLabels = tag_dataset(test_data)
            xx = []
            for x in correctLabels:
                for y in x:
                    xx.append(y)
            yy = []
            for x in predLabels:
                for y in x:
                    yy.append(y)
            acc = accuracy_score(xx, yy)
            print(acc)
            if acc > macc:
                macc = acc
                model.save('model_dial_afterwikiud_all_words_embed.h5')
                print("save_model")
            print(classification_report(xx, yy, target_names=list(idx_ud2Label.values())))#['LOC','O', 'ORG','PER']))
    return macc

In [ ]:
macc = train(train_data_ud, test_data_ud, model)

In [ ]:
model = load_model('model_wiki_afterud_all_words_embed.h5')

In [ ]:
macc = train(train_data_dial, test_data_dial, model)

In [ ]:
def get_result(dataset, model):
    correctLabels = []
    predLabels = []
    #for tokens, casing, label, char in dataset:
    for tokens, casing, label in dataset:    
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        #char = np.asarray([char])
        #pred = model.predict_classes([tokens, casing, char], verbose=False)[0]
        pred = model.predict_classes([tokens, casing], verbose=False)[0] 
        labels = [x[0] for x in label]
        correctLabels.append(labels)
        predLabels.append(pred)
    yy = []
    for x in predLabels:
        for y in x:
            yy.append(y)
    xx = []
    for x in correctLabels:
        for y in x:
            xx.append(y)
    acc = accuracy_score(xx, yy)
    print(acc)
    print(classification_report(xx, yy, target_names=list(idx2Label.values())))#['LOC','O', 'ORG','PER']))

In [ ]:
model = load_model('model_dial_test.h5')

In [ ]:
get_result(test_data_dial, model)

In [ ]:
def pop_layer(model):
    if not model.outputs:
        raise Exception('Sequential model cannot be popped: model is empty.')

    model.layers.pop()
    if not model.layers:
        model.outputs = []
        model.inbound_nodes = []
        model.outbound_nodes = []
    else:
        model.layers[-1].outbound_nodes = []
        model.outputs = [model.layers[-1].output]
    model.built = False
pop_layer(model)
model.add(TimeDistributed(Dense(len(label2Idx)-1, activation='softmax')))

sgd = SGD(lr=0.1, decay=1e-7, momentum=0.0, nesterov=False, clipvalue=3) 

model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd)

model.summary()

In [ ]:
train(train_data_dial, test_data_dial, model)

In [ ]:
def get_dataset(dataset):
    correctLabels = []
    predLabels = []
    for tokens, casing, labels in dataset:    
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        pred = model.predict_classes([tokens, casing], verbose=False)[0] 
        #labels = [x[0] for x in label]
        correctLabels.append(labels)
        predLabels.append(pred)
    yy = []
    for x in predLabels:
        for y in x:
            yy.append(y)
    xx = []
    for x in correctLabels:
        for y in x:
            xx.append(y)
    for x in range(0, len(xx)):
        xx[x][0] = yy[x]
    return xx


In [ ]:
df_test_t = pd.read_csv('../data/Dialog_test/NER_testset.targets.csv', sep=';')
df_test = pd.read_csv('../data/Dialog_test/NER_testset.features.csv', sep=';')
df_test = df_test[['pos','link', 'len', 'lemma', 'isupper', 'istitle', 'islower', 'isdigit', 'isalpha', 'isalnum', 'isLastWord', 'isFirstWord', 'grm', 'posStartInText']]

In [ ]:
#dialog -------------------- dialog
y_t = get_dataset(test_data_dial)
allINeed = pd.DataFrame(y_t, columns=["mark",  "posStartInText", "FileName", "len"])
dialog_report(allINeed)